In [45]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from sklearn.metrics import roc_curve
import statsmodels.api as sm
font_name = fm.FontProperties(fname='C:\Windows/Fonts/malgun.ttf').get_name()
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"] = False

import pymysql, math, time
from sklearn import preprocessing
import urllib

from IPython.display import Image

In [2]:
change_bright = pd.read_excel("data/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D, E, F, G, H, S, T, U, V, W, X")
change_bright.rename(columns={change_bright.columns[0]:"sex",
                        change_bright.columns[1]:"age_class",
                        change_bright.columns[2]:"definition",
                        change_bright.columns[3]:"need",
                        change_bright.columns[4]:"purpose",
                        change_bright.columns[5]:"interest",
                        change_bright.columns[6]:"comp",
                        change_bright.columns[7]:"triangle1",
                        change_bright.columns[8]:"triangle2",
                        change_bright.columns[9]:"doubleness1",
                        change_bright.columns[10]:"doubleness2",
                        change_bright.columns[11]:"doubleness3"}, inplace=True)

In [3]:
for i in range(int(change_bright.shape[0])):
    # sex : Male, Female 리스트 담기
    if change_bright["sex"][i] == "남성":
        change_bright["sex"][i] = "Male"
    else:
        change_bright["sex"][i] = "Female"
        
    # age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if change_bright["age_class"][i] == "10대":
        change_bright["age_class"][i] = "10"
    elif change_bright["age_class"][i] == "20대":
        change_bright["age_class"][i] = "20"
    elif change_bright["age_class"][i] == "30대":
        change_bright["age_class"][i] = "30"
    elif change_bright["age_class"][i] == "40대":
        change_bright["age_class"][i] = "40"
    else:
        change_bright["age_class"][i] = "50"
    
    # definition : bright, harmony로 리스트 담기
    if ("생기" in change_bright["definition"][i]):
        change_bright["definition"][i] = "bright"
    else:
        change_bright["definition"][i] = "harmony"
    
    # need :  bright, harmony로 리스트 담기
    if ("생기" in change_bright["need"][i]):
        change_bright["need"][i] = "bright"
    else:
        change_bright["need"][i] = "harmony"
    
    # purpose : clothes, cosmetic, etc로 리스트 담기
    if change_bright["purpose"][i] == "옷을 구매하는 경우에 활용":
        change_bright["purpose"][i] = "clothes"
    elif change_bright["purpose"][i] == "화장하는 경우에 활용":
        change_bright["purpose"][i] = "cosmetic"
    else:
        change_bright["purpose"][i] = "etc"

    # columns :1, 2, 3, 4 ,5 - 72, 144, origin, 216, 288
columns = ["comp", "triangle1", "triangle2", "doubleness1", "doubleness2", "doubleness3"]
for i in range(int(change_bright.shape[0])):
    for col in columns:
            if change_bright[col][i] == "①":
                change_bright[col][i] = "1"
            elif change_bright[col][i] == "②":
                change_bright[col][i] = "2"
            elif change_bright[col][i] == "③":
                change_bright[col][i] = "3"
            elif change_bright[col][i] == "④":
                change_bright[col][i] = "4"
            else:
                change_bright[col][i] = "5"

C:\Users\acorn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\acorn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\acorn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [4]:
change_bright

,sex,age_class,definition,need,purpose,interest,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3
0,Male,20,harmony,harmony,clothes,5,4,5,2,3,2,2
1,Female,20,bright,harmony,clothes,5,1,1,2,2,3,2
2,Female,20,bright,bright,clothes,3,2,4,1,2,1,2
3,Female,20,harmony,harmony,clothes,3,5,5,4,5,4,1
4,Female,20,bright,bright,etc,4,4,2,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
112,Female,30,harmony,harmony,cosmetic,4,4,2,2,4,1,3
113,Male,30,harmony,bright,cosmetic,2,1,4,5,2,4,3
114,Female,20,bright,harmony,etc,4,4,5,4,1,2,5
115,Male,20,bright,harmony,clothes,2,3,2,5,4,5,5


In [6]:
# 기본항목 전처리 결과 저장
change_bright.to_csv("data/change_bright.csv")

In [17]:
# 불러오기
df = pd.read_csv("data/change_bright.csv")

In [18]:
df

,Unnamed: 0,sex,age_class,definition,need,purpose,interest,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3
0,0,Male,20,harmony,harmony,clothes,5,4,5,2,3,2,2
1,1,Female,20,bright,harmony,clothes,5,1,1,2,2,3,2
2,2,Female,20,bright,bright,clothes,3,2,4,1,2,1,2
3,3,Female,20,harmony,harmony,clothes,3,5,5,4,5,4,1
4,4,Female,20,bright,bright,etc,4,4,2,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,112,Female,30,harmony,harmony,cosmetic,4,4,2,2,4,1,3
113,113,Male,30,harmony,bright,cosmetic,2,1,4,5,2,4,3
114,114,Female,20,bright,harmony,etc,4,4,5,4,1,2,5
115,115,Male,20,bright,harmony,clothes,2,3,2,5,4,5,5


### 답과 관련된 변수 찾기

In [12]:
# 전처리가 필요없는 컬럼
cols_to_keep = ['comp', 'triangle1', 'triangle2','doubleness1','doubleness2','doubleness3']

In [20]:
dummy_sex = pd.get_dummies(df['sex'], prefix='sex')
dummy_sex.head()

,sex_Female,sex_Male
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0


In [21]:
dummy_def = pd.get_dummies(df['definition'], prefix='definition')
dummy_def.head()

,definition_bright,definition_harmony
0,0,1
1,1,0
2,1,0
3,0,1
4,1,0


In [22]:
dummy_need = pd.get_dummies(df['need'], prefix='need')
dummy_need.head()

,need_bright,need_harmony
0,0,1
1,0,1
2,1,0
3,0,1
4,1,0


In [23]:
dummy_pur = pd.get_dummies(df['purpose'], prefix='purpose')
dummy_pur.head()

,purpose_clothes,purpose_cosmetic,purpose_etc
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,0,0,1


In [24]:
dummy_int = pd.get_dummies(df['interest'], prefix='interest')
dummy_int.head()

,interest_1,interest_2,interest_3,interest_4,interest_5
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,0,1,0


### 셀병합

In [36]:
df1 = df[cols_to_keep].join(dummy_sex.loc[:, 'sex_Male':])
df1.head()

,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3,sex_Male
0,4,5,2,3,2,2,1
1,1,1,2,2,3,2,0
2,2,4,1,2,1,2,0
3,5,5,4,5,4,1,0
4,4,2,1,1,2,2,0


In [38]:
df1 = df1.join(dummy_def.loc[:, 'definition_harmony'])
df1.head()

,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3,sex_Male,definition_harmony
0,4,5,2,3,2,2,1,1
1,1,1,2,2,3,2,0,0
2,2,4,1,2,1,2,0,0
3,5,5,4,5,4,1,0,1
4,4,2,1,1,2,2,0,0


In [39]:
df1 = df1.join(dummy_need.loc[:, 'need_harmony'])
df1.head()

,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3,sex_Male,definition_harmony,need_harmony
0,4,5,2,3,2,2,1,1,1
1,1,1,2,2,3,2,0,0,1
2,2,4,1,2,1,2,0,0,0
3,5,5,4,5,4,1,0,1,1
4,4,2,1,1,2,2,0,0,0


In [40]:
df1 = df1.join(dummy_pur.loc[:, 'purpose':])
df1.head()

,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3,sex_Male,definition_harmony,need_harmony,purpose_clothes,purpose_cosmetic,purpose_etc
0,4,5,2,3,2,2,1,1,1,1,0,0
1,1,1,2,2,3,2,0,0,1,1,0,0
2,2,4,1,2,1,2,0,0,0,1,0,0
3,5,5,4,5,4,1,0,1,1,1,0,0
4,4,2,1,1,2,2,0,0,0,0,0,1


In [59]:
df1 = df1.join(dummy_int.loc[:, 'interest':])
df1.head()

KeyError: 'interest'

In [50]:
# comp를 제외한 셀 추출
train_cols = df1.columns[6:]
train_cols

Index(['sex_Male', 'definition_harmony', 'need_harmony', 'purpose_clothes',
       'purpose_cosmetic', 'purpose_etc', 'interest_1', 'interest_2',
       'interest_3', 'interest_4', 'interest_5'],
      dtype='object')

In [48]:
# 결측치 확인
np.isnan(df1[train_cols])

,sex_Male,definition_harmony,need_harmony,purpose_clothes,purpose_cosmetic,purpose_etc,interest_1,interest_2,interest_3,interest_4,interest_5
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
112,False,False,False,False,False,False,False,False,False,False,False
113,False,False,False,False,False,False,False,False,False,False,False
114,False,False,False,False,False,False,False,False,False,False,False
115,False,False,False,False,False,False,False,False,False,False,False


In [55]:
df1

,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3,sex_Male,definition_harmony,need_harmony,purpose_clothes,purpose_cosmetic,purpose_etc,interest_1,interest_2,interest_3,interest_4,interest_5
0,4,5,2,3,2,2,1,1,1,1,0,0,0,0,0,0,1
1,1,1,2,2,3,2,0,0,1,1,0,0,0,0,0,0,1
2,2,4,1,2,1,2,0,0,0,1,0,0,0,0,1,0,0
3,5,5,4,5,4,1,0,1,1,1,0,0,0,0,1,0,0
4,4,2,1,1,2,2,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,4,2,2,4,1,3,0,1,1,0,1,0,0,0,0,1,0
113,1,4,5,2,4,3,1,1,0,0,1,0,0,1,0,0,0
114,4,5,4,1,2,5,0,0,1,0,0,1,0,0,0,1,0
115,3,2,5,4,5,5,1,0,1,1,0,0,0,1,0,0,0


In [51]:
model = sm.Logit(df1['comp'], df1[train_cols]).fit()
model.summary2()

ValueError: endog must be in the unit interval.

In [52]:
np.random.seed(0)
x1 = np.random.rand(20) + 10
x2 = np.random.rand(20) * 10
y = x1 + 2 * x2 + np.random.randn(20)
df4 = pd.DataFrame(np.array([x1, x2, y]).T, columns=["x1", "x2", "y"])

In [53]:
df4

,x1,x2,y
0,10.548814,9.786183,29.233395
1,10.715189,7.991586,24.717564
2,10.602763,4.614794,19.484438
3,10.544883,7.805292,26.311816
4,10.423655,1.182744,14.019434
5,10.645894,6.399210,24.646694
6,10.437587,1.433533,12.917326
7,10.891773,9.446689,29.482849
8,10.963663,5.218483,20.352076
9,10.383442,4.146619,17.256662


In [57]:
dfy = df1.iloc[:, -1]
dfX = sm.add_constant(df1[train_cols])
model1 = sm.OLS(dfy, dfX)

print(model1.fit().summary())

                            OLS Regression Results                            
Dep. Variable:             interest_5   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.169e+31
Date:                Fri, 13 Mar 2020   Prob (F-statistic):               0.00
Time:                        14:43:13   Log-Likelihood:                 4049.1
No. Observations:                 117   AIC:                            -8078.
Df Residuals:                     107   BIC:                            -8051.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.1304    3.7

In [58]:
model2 = LogisticRegression().fit(df1[train_cols], df1['comp'])

NameError: name 'LogisticRegression' is not defined